In [1]:
%%javascript 
// To get notebook's full path
var kernel = Jupyter.notebook.kernel; 
var command = ["notebookPath = ",
               "'", window.document.body.dataset.notebookPath, "'" ].join('')
//alert(command)
kernel.execute(command)
var command = ["notebookName = ",
               "'", window.document.body.dataset.notebookName, "'" ].join('')
//alert(command)
kernel.execute(command)

<IPython.core.display.Javascript object>

In [2]:
# Inputs:
# - X: 310 x 240 px image, 3 channels.
# Outputs:
# - y: 3 (or 6) coefficients for polynomial.

import os, sys

dir_path = os.path.dirname(os.path.abspath(notebookName))
print(dir_path)

# Path to Computer root directory
ROOT_DIR = os.path.realpath(os.path.join(dir_path, '..'))

sys.path.append(ROOT_DIR)
from libraries.helpers import configuration, prepare_initial_transformation
from libraries.find_lines_sliding_windows import FindLinesSlidingWindows
from libraries.line_helpers import annotate_with_lines

import glob
import cv2
import numpy as np
import pdb

config = configuration()

# Make sure the target shape is the same with the one in driver/main.py
# i.e. look for cams setup with variable CAP_PROP_FRAME_WIDTH and CAP_PROP_FRAME_HEIGHT.
TARGET_WIDTH = config['target_width']
TARGET_HEIGHT = config['target_height']
TARGET_CROP = config['target_crop']
STEER_MIN = config['steer_min']
STEER_MAX = config['steer_max']
CHANNELS = config['channels']
NORMALIZE = config['normalize']

mtx, dist, M, Minv = prepare_initial_transformation(
    os.path.join(ROOT_DIR, 'calibrations', 'cal-elp.p'), TARGET_HEIGHT, TARGET_WIDTH)

/home/jay/Self-Driving-RC/Computer/experiments


Using TensorFlow backend.


In [ ]:
# Create training data

def create_labels(image_paths, function, results_dir=None):
    """ Create labels from given list of images
    
    Args:
        - image_paths: List of paths of 3 channels images.
        - results_dir: If set, annotated images will be saved here.
    """
    y = []
    for image_path in image_paths:
        img = cv2.imread(image_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        fits = function(img)
        if 'left' in fits and len(fits['left']) > 0:
            label = (True, *fits['left'][0]['poly'])
        else:
            # These zeroes may create problems later. Write custom loss
            # function when that happens.
            label = (False, 0, 0, 0)
        y.append(label)
        
        if results_dir:
            # Save annotated images.
            os.makedirs(results_dir, exist_ok=True)
            undist = cv2.undistort(img, mtx, dist, None, mtx)
            warped = cv2.warpPerspective(undist, M, (TARGET_WIDTH, TARGET_HEIGHT))

            annotated = annotate_with_lines(warped, {'left': fits['left']})
            
            annotated = cv2.warpPerspective(annotated, Minv, (TARGET_WIDTH, TARGET_HEIGHT))

            filename = os.path.basename(image_path)
            filepath = os.path.join(results_dir, filename)
            cv2.imwrite(filepath, annotated)
    return (y)

# There are multiple ways to find the polynomials that will have different coverages for each.

def find_poly1(img_raw, height, width, crop, mtx, dist, M):

    img = cv2.cvtColor(img_raw, cv2.COLOR_BGR2HSV)[:, :, 1]
    img = cv2.Sobel(img, -1, 1, 0, ksize=3)
    img = img > 127
    
    img1 = cv2.cvtColor(img_raw, cv2.COLOR_BGR2HSV)[:, :, 2]
    img1 = cv2.Sobel(img1, -1, 0, 1, ksize=3)
    img1 = img1 > 127
    
    img2 = cv2.cvtColor(img_raw, cv2.COLOR_BGR2HSV)
    img2 = cv2.inRange(img2, (38, 61, 112), (139, 255, 255))
    img2 = img2 > 25.5

    final_img = (img==1) | (img1==1) | (img2==1)
    
    f3 = np.stack((final_img, final_img, final_img), axis=2)
    f3 = (f3 * 255.0).astype(np.uint8)

    undist = cv2.undistort(f3, mtx, dist, None, mtx)
    warped = cv2.warpPerspective(undist, M, (width, height))
    warped = warped[:, :, 0]

    finder = FindLinesSlidingWindows(nwindows=30, window_minpix=20, subsequent_search_margin=30,
                                     lr_start_slack=0.1, debug=False, error_top_percentile=75,
                                     center_importance=4,
                                     error_threshold=30, window_patience=7, window_empty_px=5)
    fits = finder.process(warped)
    return fits

def annotate_with_lines(background, lines, keep_background=True, Minv=None):
    """ Annotate a background with lines. """    
    ploty = [i for i in range(background.shape[0])]
    # Create an image to draw the lines on
    if keep_background:
        canvas = np.array(background, dtype=np.uint8)
    else:
        canvas = np.zeros_like(background).astype(np.uint8)

    # If grey image, convert to three channels
    if len(canvas.shape) == 2 or canvas.shape[2] == 1:
        canvas = np.stack((canvas, canvas, canvas), axis=2)
        canvas = (canvas * 255.0).astype(np.uint8)
    
    if 'left' in lines:
        for line in lines['left']:
            if line is not None:
                fitx = line['x']
                pts = np.array([np.transpose(np.vstack([fitx, ploty]))], np.int32)
                cv2.polylines(canvas, np.int_([pts]), False, (255, 255, 0), thickness=3)
            
    if 'right' in lines:
        for line in lines['right']:
            if line is not None:
                fitx = line['x']
                pts = np.array([np.transpose(np.vstack([fitx, ploty]))], np.int32)
                cv2.polylines(canvas, np.int_([pts]), False, (0, 255, 0), thickness=3)
        
    return canvas

def find_poly2(image):
    finder = SlidingWindow(param1)
    return processed_image

image_paths = glob.glob("/home/jay/Self-Driving-RC-Data/recorded-2017-06-11-training/recorded/*.jpg")

y = create_labels(image_paths[:],
                  lambda img: find_poly1(img, TARGET_HEIGHT, TARGET_WIDTH, TARGET_CROP, mtx, dist, M),
                  results_dir="/home/jay/Self-Driving-RC-Data/recorded-2017-06-11-training/poly")

In [19]:
y[1]

(True, -0.006969107601466607, 0.28172161740877882, 338.11055318359342)

In [22]:
# Create model

images = load_images

def train(images, y):
    pass

NameError: name 'load_images' is not defined